In [73]:
import numpy as np
import pandas as pd
import sys
import os
from sklearn.preprocessing import OrdinalEncoder, StandardScaler

# low-variance feature removal
from sklearn.feature_selection import VarianceThreshold

# feature importance
from sklearn.ensemble import RandomForestClassifier

# Add project root to sys.path
sys.path.append(os.path.abspath("..")) # go one level up

from src.utils.paths import CLEAN_DATA_DIR
from src.utils.io import load_csv

df = load_csv(CLEAN_DATA_DIR / "cleaned_telco_churn.csv")
df_fe = df.copy()

In [74]:
pd.set_option('display.max_columns', None)
df_fe.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_zscore,tenure_outlier_z,MonthlyCharges_zscore,MonthlyCharges_outlier_z,TotalCharges_zscore,TotalCharges_outlier_z,tenure_outlier_iqr,MonthlyCharges_outlier_iqr,TotalCharges_outlier_iqr,outlier_iso
3451,5397-TUPSH,Male,1,Yes,No,1.328659,Yes,No,Fiber optic,No,No,No,No,No,No,One year,Yes,Bank transfer (automatic),0.205674,1.003608,No,1.328659,0,0.205674,0,1.003608,0,0,0,0,0
1155,6870-ZWMNX,Male,0,Yes,No,1.532261,Yes,Yes,DSL,No,Yes,Yes,Yes,Yes,No,Two year,No,Credit card (automatic),0.376839,1.316640,No,1.532261,0,0.376839,0,1.316640,0,0,0,0,0
784,5125-CNDSP,Male,0,No,Yes,1.247218,No,No phone service,DSL,Yes,Yes,No,Yes,No,No,One year,Yes,Bank transfer (automatic),-0.803037,0.135549,No,1.247218,0,-0.803037,0,0.135549,0,0,0,0,0
5644,9444-JTXHZ,Male,0,Yes,No,-1.277445,Yes,No,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Electronic check,0.380163,-0.973779,Yes,-1.277445,0,0.380163,0,-0.973779,0,0,0,0,0
4112,6627-CFOSN,Female,0,No,No,1.084337,Yes,Yes,Fiber optic,Yes,Yes,Yes,No,No,Yes,Month-to-month,Yes,Bank transfer (automatic),1.152899,1.475330,No,1.084337,0,1.152899,0,1.475330,0,0,0,0,0


#### **1. Feature Construction**

**1.1 Total Service Count:**

In [75]:
service_cols = [
    'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup',
    'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies'
]

df_fe['service_count'] = df_fe[service_cols].eq('Yes').sum(axis=1)

**1.2 Monthly Charge Buckets**

In [76]:
df_fe['charges_bucket'] = pd.cut(
    df_fe['MonthlyCharges'],
    bins=[0,35, 70, 100, 150],
    labels=['Low', 'Medium', 'High', 'Very High']
)

**1.3 Total charges per tenure ratio**

In [77]:
# For each customer, if tenure is not 0, divide TotalCharges by tenure and store the result in charge_per_month.
# If tenure is 0, set charge_per_month to 0.

df_fe['charges_per_month'] = np.where(
    df_fe['tenure'] !=0,
    df_fe['TotalCharges'] / df_fe['tenure'],
    0
)

**1.4 Active User (Internet + Streaming)**

In [ ]:
# Is active user (internet + streaming)
# Mark 1 if customer has InternetService and subscribes to StreamingTV, else 0
df_fe['is_active_user'] = ((df_fe['InternetService'] != 'No') & (df_fe['StreamingTV'] == 'Yes')).astype(int)

**1.5 Senior Citizen**

In [79]:
# Senior citizen as binary
# Convert SeniorCitizen column to 1 if senior (1), else 0
df_fe['is_senior'] = df_fe['SeniorCitizen'].astype(int)


**1.6 Tenure Squared**

In [80]:
# Tenure squared (classic interaction feature)
# Create a new feature as the square of tenure
df_fe['tenure_sqr'] = df_fe['tenure']**2

**1.7  High monthly charge flag**

In [81]:
# Flag high bills: 1 if MonthlyCharges > median, else 0
df_fe['high_bill_flag'] = (df_fe['MonthlyCharges'] > df_fe['MonthlyCharges'].median()).astype(int)

In [82]:
df_fe.sample(5)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn,tenure_zscore,tenure_outlier_z,MonthlyCharges_zscore,MonthlyCharges_outlier_z,TotalCharges_zscore,TotalCharges_outlier_z,tenure_outlier_iqr,MonthlyCharges_outlier_iqr,TotalCharges_outlier_iqr,outlier_iso,service_count,charges_bucket,charges_per_month,is_active_user,is_senior,tenure_sqr,high_bill_flag
1052,7013-PSXHK,Female,0,No,No,0.310650,Yes,Yes,DSL,No,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,0.531386,0.409134,No,0.310650,0,0.531386,0,0.409134,0,0,0,0,0,6,Low,1.317027,1,0,0.096503,1
1454,8681-ICONS,Male,0,Yes,Yes,-0.137274,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,Yes,Mailed check,-1.466094,-0.718317,No,-0.137274,0,-1.466094,0,-0.718317,0,0,0,0,0,1,NaN,5.232710,0,0,0.018844,0
4450,4925-LMHOK,Male,0,No,No,-0.707359,Yes,Yes,DSL,No,Yes,No,Yes,No,No,Month-to-month,Yes,Credit card (automatic),-0.204790,-0.592562,Yes,-0.707359,0,-0.204790,0,-0.592562,0,0,0,0,0,4,NaN,0.837709,0,0,0.500357,0
3617,3346-BRMIS,Female,1,Yes,No,-0.585198,No,No phone service,DSL,No,No,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,-0.545458,-0.649512,Yes,-0.585198,0,-0.545458,0,-0.649512,0,0,0,0,0,3,NaN,1.109901,1,1,0.342457,0
5009,0052-DCKON,Male,0,Yes,No,1.369379,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,One year,No,Bank transfer (automatic),1.696306,2.498878,No,1.369379,0,1.696306,0,2.498878,0,0,0,0,1,8,Low,1.824826,1,0,1.875199,1


##### **2. Dropping and Mapping Columns**

In [83]:
# Dropping column = customerID not required for modeling.
df_fe = df_fe.drop(columns=['customerID'], errors='ignore')

In [84]:
# Mapping target Variable
# Churn: No -> 0,  Yes -> 1
df_fe['Churn'] = df_fe['Churn'].map({'No':0, 'Yes':1})

##### **3. Identifying Categorical columns**

In [86]:
# identifying categorical columns in dataset.
# categorical_cols = df_fe.select_dtypes(include=['object']).columns.tolist()
# print("Categorical columns:", categorical_cols)

# Ordinal columns (have natural order)
ordinal_cols = ['Contract']

# Nominal columns (no order, remaining object type columns)
nominal_cols = [c for c in df_fe.select_dtypes(include=['object']).columns if c not in ordinal_cols + ['Churn']]
print("Ordinal columns:", ordinal_cols)
print("Nominal columns:", nominal_cols)

Ordinal columns: ['Contract']
Nominal columns: ['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'PaymentMethod']


##### **4. Ordinal Encoding**

In [87]:
# Ordinal column order
ordinal_mapping = {
    'Contract': ['Month-to-month', 'One year', 'Two year']
}

# Encode ordinal columns to numbers using their defined order
# Resulting numeric values are stored in a new column named '<original_column>_encoded'
for col, cat_order in ordinal_mapping.items():
    encoder = OrdinalEncoder(categories=[cat_order])
    df_fe[col + '_encoded'] = encoder.fit_transform(df_fe[[col]])


# Dropping original ordinal column after encoding
df_fe = df_fe.drop(columns=ordinal_cols)

##### **5. Nominal / One-Hot Encoding**

In [88]:
# One-Hot Encode all nominal columns, drop first category to avoid redundancy
df_fe = pd.get_dummies(df_fe, columns=nominal_cols, drop_first=True)


print("Columns after encoding:")
print(df_fe.columns)

print("\nSample of first 5 rows:")
display(df_fe.head())

print("\nCheck for missing values:")
print(df_fe.isna().sum())

Columns after encoding:
Index(['SeniorCitizen', 'tenure', 'MonthlyCharges', 'TotalCharges', 'Churn',
       'tenure_zscore', 'tenure_outlier_z', 'MonthlyCharges_zscore',
       'MonthlyCharges_outlier_z', 'TotalCharges_zscore',
       'TotalCharges_outlier_z', 'tenure_outlier_iqr',
       'MonthlyCharges_outlier_iqr', 'TotalCharges_outlier_iqr', 'outlier_iso',
       'service_count', 'charges_bucket', 'charges_per_month',
       'is_active_user', 'is_senior', 'tenure_sqr', 'high_bill_flag',
       'Contract_encoded', 'gender_Male', 'Partner_Yes', 'Dependents_Yes',
       'PhoneService_Yes', 'MultipleLines_No phone service',
       'MultipleLines_Yes', 'InternetService_Fiber optic',
       'InternetService_No', 'OnlineSecurity_No internet service',
       'OnlineSecurity_Yes', 'OnlineBackup_No internet service',
       'OnlineBackup_Yes', 'DeviceProtection_No internet service',
       'DeviceProtection_Yes', 'TechSupport_No internet service',
       'TechSupport_Yes', 'StreamingTV_No in

,SeniorCitizen,tenure,MonthlyCharges,TotalCharges,Churn,tenure_zscore,tenure_outlier_z,MonthlyCharges_zscore,MonthlyCharges_outlier_z,TotalCharges_zscore,TotalCharges_outlier_z,tenure_outlier_iqr,MonthlyCharges_outlier_iqr,TotalCharges_outlier_iqr,outlier_iso,service_count,charges_bucket,charges_per_month,is_active_user,is_senior,tenure_sqr,high_bill_flag,Contract_encoded,gender_Male,Partner_Yes,Dependents_Yes,PhoneService_Yes,MultipleLines_No phone service,MultipleLines_Yes,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,PaperlessBilling_Yes,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,0,-1.277445,-1.160323,-0.994242,0,-1.277445,0,-1.160323,0,-0.994242,0,0,0,0,0,1,NaN,0.778305,0,0,1.631865,0,0.0,False,True,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,True,False,True,False
1,0,0.066327,-0.259629,-0.173244,0,0.066327,0,-0.259629,0,-0.173244,0,0,0,0,0,3,NaN,-2.611953,0,0,0.004399,0,1.0,True,False,False,True,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True
2,0,-1.236724,-0.362660,-0.959674,1,-1.236724,0,-0.362660,0,-0.959674,0,0,0,0,0,3,NaN,0.775981,0,0,1.529487,0,0.0,True,False,False,True,False,False,False,False,False,True,False,True,False,False,False,False,False,False,False,False,True,False,False,True
3,0,0.514251,-0.746535,-0.194766,0,0.514251,0,-0.746535,0,-0.194766,0,0,0,0,0,3,NaN,-0.378737,0,0,0.264455,0,1.0,True,False,False,False,True,False,False,False,False,True,False,False,False,True,False,True,False,False,False,False,False,False,False,False
4,0,-1.236724,0.197365,-0.940470,1,-1.236724,0,0.197365,0,-0.940470,0,0,0,0,0,1,Low,0.760452,0,0,1.529487,1,0.0,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False



Check for missing values:
SeniorCitizen                               0
tenure                                      0
MonthlyCharges                              0
TotalCharges                                0
Churn                                       0
tenure_zscore                               0
tenure_outlier_z                            0
MonthlyCharges_zscore                       0
MonthlyCharges_outlier_z                    0
TotalCharges_zscore                         0
TotalCharges_outlier_z                      0
tenure_outlier_iqr                          0
MonthlyCharges_outlier_iqr                  0
TotalCharges_outlier_iqr                    0
outlier_iso                                 0
service_count                               0
charges_bucket                           3120
charges_per_month                           0
is_active_user                              0
is_senior                                   0
tenure_sqr                                  0
high_bi

##### **6. Feature Selection / Reduction**

In [90]:
#---------------------------------------
# Separate Feature and Target
target_col = 'Churn'
X = df_fe.drop(columns=[target_col])
y = df_fe[target_col]

#---------------------------------------
# Remove Zero-Variance Feature
constant_cols = [col for col in X.columns if X[col].nunique()==1]
X = X.drop(columns=constant_cols)
print("After Variance Threshold:", X.shape)

#---------------------------------------
# REMOVE HIGHLY CORRELATED FEATURES

# Compute absolute correlation matrix to see strength of relationships
corr_matrix = X.corr().abs()

# Keep only the upper triangle of the correlation matrix to avoid duplicate checks
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))

# Find columns with correlation > 0.85 with any other feature
to_drop_corr = [col for col in upper.columns if any(upper[col] > 0.85)]

# Drop highly correlated columns to reduce redundancy and multicollinearity
X = X.drop(columns=to_drop_corr)

print("After Correlation Filtering:", X.shape)


#---------------------------------------
# MODEL-BASED FEATURE IMPORTANCE (RANDOM FOREST)

# Train a Random Forest to compute feature importances,
# then retain only the top N most predictive features.
rf = RandomForestClassifier(n_estimators=300, random_state=42, n_jobs=-1)
rf.fit(X, y)

# Extract feature importance scores
importances = pd.Series(rf.feature_importances_, index=X.columns)

# Select top N features based on importance
top_features = importances.sort_values(ascending=False).head().index


# Create final dataset containing only top features
X_final = X[top_features]

print("After Model-Based Selection:", X_final.shape)
print("\nTop Selected Features:\n", X_final.columns.tolist())


After Variance Threshold: (7043, 39)
After Correlation Filtering: (7043, 26)
After Model-Based Selection: (7043, 5)

Top Selected Features:
 ['TotalCharges', 'charges_per_month', 'MonthlyCharges', 'tenure', 'Contract_encoded']
